In [1]:
import pandas as pd
import numpy as np

In [2]:
df_1=pd.read_csv('MELBOURNE_HOUSE_PRICES_LESS.csv')
df_2=pd.read_csv('Melbourne_housing_FULL.csv')

In [3]:
df=pd.merge(df_1,df_2,how='outer')

In [4]:
df=df.dropna()

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:

# Feature engineering
df['Age'] = 2023 - df['YearBuilt']

In [7]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['Suburb'] = lab.fit_transform(df['Suburb'])

In [8]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['Address'] = lab.fit_transform(df['Address'])

In [9]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['Type'] = lab.fit_transform(df['Type'])

In [10]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['Method'] = lab.fit_transform(df['Method'])

In [11]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['SellerG '] = lab.fit_transform(df['SellerG'])

In [12]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['Date'] = lab.fit_transform(df['Date'])

In [13]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['Regionname'] = lab.fit_transform(df['Regionname'])

In [14]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['CouncilArea'] = lab.fit_transform(df['CouncilArea'])

In [15]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['SellerG '] = lab.fit_transform(df['SellerG'])

In [16]:
df.drop(['SellerG'],axis=1, inplace=True)

In [17]:
# Finalize the data
X = df.drop(['Price'], axis=1)
y = df['Price']


In [18]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

In [19]:
class DecisionTreeRegressor:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        
    def mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)
    
    def split(self, X, y, idx, split_val):
        left_idx = np.argwhere(X[:, idx] <= split_val).flatten()
        right_idx = np.argwhere(X[:, idx] > split_val).flatten()
        
        if len(left_idx) == 0 or len(right_idx) == 0:
            return None, None
        
        left_X, left_y = X[left_idx], y[left_idx]
        right_X, right_y = X[right_idx], y[right_idx]
        
        left_mse, right_mse = self.mse(left_y), self.mse(right_y)
        return left_X, left_y, right_X, right_y, left_mse + right_mse
    
    def find_best_split(self, X, y):
        best_idx, best_val, best_mse = None, None, np.inf
        
        for idx in range(X.shape[1]):
            for val in np.unique(X[:, idx]):
                left_X, left_y, right_X, right_y, mse = self.split(X, y, idx, val)
                if mse is not None and mse < best_mse:
                    best_idx, best_val, best_mse = idx, val, mse
        
        return best_idx, best_val, best_mse
    
    def fit(self, X, y, depth=0):
        if self.max_depth is not None and depth >= self.max_depth:
            self.left, self.right = None, None
            self.prediction = np.mean(y)
            return
        
        idx, val, mse = self.find_best_split(X, y)
        if idx is None:
            self.left, self.right = None, None
            self.prediction = np.mean(y)
            return
        
        left_X, left_y, right_X, right_y, _ = self.split(X, y, idx, val)
        self.feature_idx = idx
        self.split_val = val
        self.left = DecisionTreeRegressor(max_depth=self.max_depth)
        self.right = DecisionTreeRegressor(max_depth=self.max_depth)
        self.left.fit(left_X, left_y, depth=depth+1)
        self.right.fit(right_X, right_y, depth=depth+1)
    
    def predict(self, X):
        if self.left is None and self.right is None:
            return self.prediction
        
        if X[self.feature_idx] <= self.split_val:
            return self.left.predict(X)
        else:
            return self.right.predict(X)

In [20]:
class RandomForestRegressor:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def fit(self, X, y):
        for i in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X, y)
            self.trees.append(tree)

    def predict(self, X):
        predictions = []
        for tree in self.trees:
            predictions.append(tree.predict(X))
        return np.mean(predictions, axis=0)

In [21]:
from sklearn.model_selection import train_test_split
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
from sklearn.tree import DecisionTreeRegressor

In [23]:
rf = RandomForestRegressor()

In [24]:
rf.fit(X_train, y_train)

In [25]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE: ", mse)
print("R-squared: ", r2)

MSE:  174689844017.53775
R-squared:  0.6329911470417242
